https://www.kaggle.com/aashita/nyt-comments

In [20]:
from string import punctuation
print(punctuation)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [3]:
df =pd.read_csv("nyt.csv")
df.head()['snippet']

0    “I understand that they could meet with us, pa...
1    The agency plans to publish a new regulation T...
2    What’s it like to eat at the second incarnatio...
3    President Trump welcomed President Emmanuel Ma...
4    Alek Minassian, 25, a resident of Toronto’s Ri...
Name: snippet, dtype: object

In [6]:
# df.info()
df['headline'].isnull().sum()

0

In [8]:
df['headline'].values

array(['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
       'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
       'The New Noma, Explained', ...,
       'Gen. Michael Hayden Has One Regret: Russia',
       'There Is Nothin’ Like a Tune', 'Unknown'], dtype=object)

In [41]:
headline=[]
headline.extend(list(df.headline.values))
headline[:10]


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [42]:
headline =[n for n in headline if n !='Unknown']
len(headline)

1214

In [16]:
# "' ".encode('utf-8').decode('ascii', 'ignore')

"' "

In [43]:
def repreprocessing(s):
    s=s.encode('utf8').decode('ascii', 'ignore')
    #s="Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell"
    return ''.join(c for c in s if c not in punctuation).lower()
    #s에 전달된 뉴스기사제목에 대해 전처리(점 제거, 소문자)
    #return result    
text=[repreprocessing(x) for x in headline]
#                 <-2->  <-     1        ->
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [44]:
t=Tokenizer()
t.fit_on_texts(text)

In [45]:
vocabSize=len(t.word_index)+1 #3493 +1 단어

In [46]:
sequences = []
for line in text:
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence =encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [47]:
# ['former nfl cheerleaders settlement offer 1 and a meeting with goodell
#    99    269    371         1115
#t.word_counts
indextoWord={}
for k, v in t.word_index.items(): #{'the':1}
    indextoWord[v]=k  #{1:'the'}

In [48]:
maxLen=max(len(s) for s in sequences)
print(maxLen) #24

24


In [49]:
sequences=pad_sequences(sequences, maxlen=maxLen, 
                        padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [50]:
x=sequences[:,:-1]
y=sequences[:,-1]


In [51]:
print(x[0])
print(y[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 99]
269


In [52]:
y=to_categorical(y, num_classes=vocabSize)

In [53]:
y.shape

(7803, 3494)

In [54]:
model= Sequential()
model.add(Embedding(vocabSize, 10, input_length=maxLen-1))
#입력 데이터의 길이는 maxlen 에서  y를 뺀 
model.add(SimpleRNN(32))
model.add(Dense(vocabSize, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x, y, epochs=200, batch_size=10)

Epoch 1/200
781/781 [==============================] - 3s 3ms/step - loss: 7.6758 - accuracy: 0.0305
Epoch 2/200
781/781 [==============================] - 2s 3ms/step - loss: 7.0931 - accuracy: 0.0352
Epoch 3/200
781/781 [==============================] - 2s 3ms/step - loss: 6.8484 - accuracy: 0.0461
Epoch 4/200
781/781 [==============================] - 2s 3ms/step - loss: 6.6182 - accuracy: 0.0495
Epoch 5/200
781/781 [==============================] - 3s 3ms/step - loss: 6.3811 - accuracy: 0.0531
Epoch 6/200
781/781 [==============================] - 2s 3ms/step - loss: 6.1430 - accuracy: 0.0610
Epoch 7/200
781/781 [==============================] - 2s 3ms/step - loss: 5.9230 - accuracy: 0.0683
Epoch 8/200
781/781 [==============================] - 2s 3ms/step - loss: 5.7145 - accuracy: 0.0761
Epoch 9/200
781/781 [==============================] - 2s 3ms/step - loss: 5.5182 - accuracy: 0.0846
Epoch 10/200
781/781 [==============================] - 2s 3ms/step - loss: 5.3295 - accura

In [57]:
def sentence_generation(model, t, cw, n):
    #문장 예측 생성
    initWord = cw #시작단어 저장
    sentence =''
    for _ in range(n):
        #4번 반복
        #_ : 변수가 없다. 
        # print(t.texts_to_sequences([cw]))
        encoded = t.texts_to_sequences([cw])[0]
        encoded =pad_sequences([encoded], maxlen=5, padding='pre')
        # print(encoded)
        result = model.predict_classes(encoded)
        # print(result)
        for word, index in t.word_index.items():
            if index==result:
                break
        cw=cw+" "+ word #"들판에 있는"
        sentence = sentence+" "+ word
    sentence=initWord+sentence
    return sentence

sentence_generation(model, t, 'you', 10)

'you little big leader of afghan if jail its long them'